VLM-GUIDED SEGMENTATION

APPROCHES À TESTER:
1. CLIP-guided U-Net: Features CLIP + U-Net
2. Text-prompted segmentation: Descriptions textuelles → masques
3. Visual-Semantic fusion: Combiner embeddings visuels + textuels

In [4]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import scipy.ndimage as ndi
from tqdm import tqdm
import pandas as pd
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

# Pour CLIP (Vision-Language Model)
try:
    import clip
    CLIP_AVAILABLE = True
    print(" CLIP disponible!")
except:
    CLIP_AVAILABLE = False
    print(" CLIP non disponible. Installation: pip install git+https://github.com/openai/CLIP.git")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🖥️ Device: {device}")

 CLIP disponible!
🖥️ Device: cpu


In [3]:
DATASET_PATH = r"C:\Users\ayoub\Downloads\Task04_Hippocampus\Task04_Hippocampus"  
PREPROCESSED_PATH = os.path.join(DATASET_PATH, "preprocessed")
RESULTS_PATH = os.path.join(DATASET_PATH, "results")
VLM_PATH = os.path.join(RESULTS_PATH, "vlm_segmentation")
os.makedirs(VLM_PATH, exist_ok=True)

print(f"✅ Configuration prête!")
print(f"📁 Résultats VLM: {VLM_PATH}")

✅ Configuration prête!
📁 Résultats VLM: C:\Users\ayoub\Downloads\Task04_Hippocampus\Task04_Hippocampus\results\vlm_segmentation


Installation et chargement de CLIP

In [5]:
"""
CLIP: Contrastive Language-Image Pre-training
Modèle pré-entraîné sur 400M paires (image, texte)
"""
if CLIP_AVAILABLE:
    # Charger CLIP pré-entraîné
    clip_model, preprocess = clip.load("ViT-B/32", device=device)
    clip_model.eval()
    
    print("✅ CLIP chargé!")
    print(f"  Architecture: ViT-B/32")
    print(f"  Image encoder: Vision Transformer")
    print(f"  Text encoder: Transformer")
    
    # Test CLIP
    test_texts = [
        "anterior hippocampus in brain MRI",
        "posterior hippocampus in brain scan",
        "background tissue in medical image"
    ]
    
    with torch.no_grad():
        text_tokens = clip.tokenize(test_texts).to(device)
        text_features = clip_model.encode_text(text_tokens)
        text_features = text_features / text_features.norm(dim=-1, keepdim=True)
    
    print(f"\n🧪 Test CLIP:")
    print(f"  Text embeddings shape: {text_features.shape}")
    print(f"  Dimension: 512 (standard CLIP)")
else:
    print("\n⚠️ CLIP non disponible. Utiliser une approche alternative.")
    print("   Installation: pip install git+https://github.com/openai/CLIP.git")

100%|███████████████████████████████████████| 338M/338M [01:09<00:00, 5.10MiB/s]


✅ CLIP chargé!
  Architecture: ViT-B/32
  Image encoder: Vision Transformer
  Text encoder: Transformer

🧪 Test CLIP:
  Text embeddings shape: torch.Size([3, 512])
  Dimension: 512 (standard CLIP)


Descriptions textuelles anatomiques :

Ces descriptions guident le modèle pour comprendre ce qu'il cherche.
Plus les descriptions sont précises, meilleure sera la segmentation!

In [6]:
# Descriptions par classe
CLASS_DESCRIPTIONS = {
    'background': [
        "background tissue in brain MRI",
        "non-hippocampus brain tissue",
        "surrounding brain structures",
        "white matter and gray matter",
        "ventricles and CSF in brain"
    ],
    'anterior': [
        "anterior hippocampus in temporal lobe",
        "head of hippocampus, curved structure",
        "anterior part of hippocampal formation",
        "rostral hippocampus near amygdala",
        "hippocampal head in coronal MRI slice"
    ],
    'posterior': [
        "posterior hippocampus in brain",
        "tail of hippocampus, elongated structure",
        "posterior part of hippocampal formation",
        "caudal hippocampus extending posteriorly",
        "hippocampal tail in sagittal view"
    ]
}

# Descriptions générales pour contexte global
CONTEXT_DESCRIPTIONS = [
    "medical brain MRI scan showing temporal lobe structures",
    "T1-weighted MRI of human brain with visible hippocampus",
    "coronal brain slice with bilateral hippocampi",
    "neuroanatomical structures in medial temporal lobe",
    "high-resolution brain imaging of limbic system"
]

print("📝 DESCRIPTIONS TEXTUELLES DÉFINIES:")
print(f"  Background: {len(CLASS_DESCRIPTIONS['background'])} descriptions")
print(f"  Anterior: {len(CLASS_DESCRIPTIONS['anterior'])} descriptions")
print(f"  Posterior: {len(CLASS_DESCRIPTIONS['posterior'])} descriptions")
print(f"  Context: {len(CONTEXT_DESCRIPTIONS)} descriptions")

print("\n💡 EXEMPLE:")
print(f"  Anterior: '{CLASS_DESCRIPTIONS['anterior'][0]}'")
print(f"  Posterior: '{CLASS_DESCRIPTIONS['posterior'][0]}'")

📝 DESCRIPTIONS TEXTUELLES DÉFINIES:
  Background: 5 descriptions
  Anterior: 5 descriptions
  Posterior: 5 descriptions
  Context: 5 descriptions

💡 EXEMPLE:
  Anterior: 'anterior hippocampus in temporal lobe'
  Posterior: 'posterior hippocampus in brain'


Encoder CLIP pour embeddings textuels

In [7]:
if CLIP_AVAILABLE:
    def get_text_embeddings(descriptions_dict, clip_model, device):
        """
        Génère les embeddings textuels moyens pour chaque classe
        
        Returns:
            text_embeddings: Tensor (num_classes, embedding_dim)
        """
        embeddings_per_class = []
        
        for class_name in ['background', 'anterior', 'posterior']:
            descriptions = descriptions_dict[class_name]
            
            # Tokenize et encoder
            tokens = clip.tokenize(descriptions).to(device)
            with torch.no_grad():
                class_embeddings = clip_model.encode_text(tokens)
                class_embeddings = class_embeddings / class_embeddings.norm(dim=-1, keepdim=True)
            
            # Moyenne des embeddings
            mean_embedding = class_embeddings.mean(dim=0)
            mean_embedding = mean_embedding / mean_embedding.norm()
            
            embeddings_per_class.append(mean_embedding)
        
        return torch.stack(embeddings_per_class)  # (3, 512)
    
    
    # Générer les embeddings
    text_embeddings = get_text_embeddings(CLASS_DESCRIPTIONS, clip_model, device)
    
    print(f"✅ Embeddings textuels générés!")
    print(f"  Shape: {text_embeddings.shape}")
    print(f"  Classes: Background, Anterior, Posterior")
    
    # Calculer similarité entre classes (doit être faible)
    similarities = torch.mm(text_embeddings, text_embeddings.t())
    print(f"\n📊 Similarités entre classes:")
    print(f"  Background ↔ Anterior: {similarities[0, 1].item():.3f}")
    print(f"  Background ↔ Posterior: {similarities[0, 2].item():.3f}")
    print(f"  Anterior ↔ Posterior: {similarities[1, 2].item():.3f}")
    print(f"  → Plus la similarité est faible, mieux les classes sont séparées!")

✅ Embeddings textuels générés!
  Shape: torch.Size([3, 512])
  Classes: Background, Anterior, Posterior

📊 Similarités entre classes:
  Background ↔ Anterior: 0.906
  Background ↔ Posterior: 0.903
  Anterior ↔ Posterior: 0.977
  → Plus la similarité est faible, mieux les classes sont séparées!


Dataset avec preprocessing CLIP

In [8]:
class HippocampusDatasetVLM(Dataset):
    def __init__(self, file_list, preprocessed_path, split='train', 
                 augment_config=None, clip_preprocess=None):
        self.file_list = file_list
        self.preprocessed_path = preprocessed_path
        self.split = split
        self.augment_config = augment_config or {}
        self.clip_preprocess = clip_preprocess
        
        # Liste des slices 2D
        self.slices = []
        for file_info in file_list:
            base_name = file_info['image'].replace('.nii.gz', '')
            img_path = os.path.join(preprocessed_path, split, f"{base_name}_img.npy")
            lbl_path = os.path.join(preprocessed_path, split, f"{base_name}_lbl.npy")
            
            if os.path.exists(img_path):
                volume = np.load(img_path)
                num_slices = volume.shape[2]
                
                for slice_idx in range(num_slices):
                    self.slices.append({
                        'volume_path': img_path,
                        'label_path': lbl_path,
                        'slice_idx': slice_idx
                    })
        
        print(f"  {split}: {len(self.slices)} slices 2D (VLM dataset)")
    
    def __len__(self):
        return len(self.slices)
    
    def __getitem__(self, idx):
        slice_info = self.slices[idx]
        
        volume = np.load(slice_info['volume_path'])
        label = np.load(slice_info['label_path'])
        
        slice_idx = slice_info['slice_idx']
        image_2d = volume[:, :, slice_idx]
        label_2d = label[:, :, slice_idx]
        
        # Augmentation
        if self.augment_config:
            if self.augment_config.get('flip', False) and np.random.rand() > 0.5:
                image_2d = np.flip(image_2d, axis=0).copy()
                label_2d = np.flip(label_2d, axis=0).copy()
            
            if self.augment_config.get('rotation', False) and np.random.rand() > 0.7:
                angle = np.random.uniform(-10, 10)
                image_2d = ndi.rotate(image_2d, angle, reshape=False, order=1)
                label_2d = ndi.rotate(label_2d, angle, reshape=False, order=0)
        
        # Format standard pour U-Net
        image_unet = torch.from_numpy(image_2d).float().unsqueeze(0)  # (1, H, W)
        
        # Format CLIP (si disponible)
        if self.clip_preprocess is not None and CLIP_AVAILABLE:
            # Convertir en RGB (3 canaux) pour CLIP
            image_rgb = np.stack([image_2d]*3, axis=-1)
            # Normaliser [0, 1]
            image_rgb = (image_rgb - image_rgb.min()) / (image_rgb.max() - image_rgb.min() + 1e-8)
            # Convertir en PIL puis appliquer preprocessing CLIP
            from PIL import Image
            image_pil = Image.fromarray((image_rgb * 255).astype(np.uint8))
            image_clip = self.clip_preprocess(image_pil)
        else:
            image_clip = None
        
        label_2d = torch.from_numpy(label_2d).long()
        
        return {
            'image_unet': image_unet,
            'image_clip': image_clip,
            'label': label_2d
        }


# Charger données
with open(os.path.join(PREPROCESSED_PATH, 'split_info.json'), 'r') as f:
    split_info = json.load(f)

train_files = split_info['train']
val_files = split_info['val']

augment_config = {'flip': True, 'rotation': True}

if CLIP_AVAILABLE:
    train_dataset_vlm = HippocampusDatasetVLM(
        train_files, PREPROCESSED_PATH, split='train',
        augment_config=augment_config, clip_preprocess=preprocess
    )
    
    val_dataset_vlm = HippocampusDatasetVLM(
        val_files, PREPROCESSED_PATH, split='val',
        augment_config=None, clip_preprocess=preprocess
    )
    
    train_loader_vlm = DataLoader(train_dataset_vlm, batch_size=8, shuffle=True, num_workers=0)
    val_loader_vlm = DataLoader(val_dataset_vlm, batch_size=8, shuffle=False, num_workers=0)
    
    print(f"\n✅ DataLoaders VLM créés")
    
    # Test
    batch = next(iter(train_loader_vlm))
    print(f"\n🧪 Test batch:")
    print(f"  image_unet: {batch['image_unet'].shape}")
    print(f"  image_clip: {batch['image_clip'].shape if batch['image_clip'] is not None else None}")
    print(f"  label: {batch['label'].shape}")


  train: 13312 slices 2D (VLM dataset)
  val: 3328 slices 2D (VLM dataset)

✅ DataLoaders VLM créés

🧪 Test batch:
  image_unet: torch.Size([8, 1, 64, 64])
  image_clip: torch.Size([8, 3, 224, 224])
  label: torch.Size([8, 64, 64])


CLIP-Guided U-Net - Architecture hybride

Architecture innovante qui combine:
- U-Net pour segmentation
- CLIP features pour guidance sémantique
- Fusion des embeddings visuels et textuels

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CLIPGuidedUNet(nn.Module):
    """
    CLIP-Guided U-Net (Corrected Version)

    - CLIP visual encoder (frozen)
    - U-Net for spatial precision
    - Visual-Semantic alignment via cosine similarity (CLIP-style)
    """

    def __init__(self, clip_model, text_embeddings, num_classes=3, base_filters=32):
        super().__init__()

        # -----------------------------
        # CLIP VISUAL ENCODER (FROZEN)
        # -----------------------------
        self.clip_visual = clip_model.visual
        for p in self.clip_visual.parameters():
            p.requires_grad = False

        self.register_buffer("text_embeddings", text_embeddings)  # (C, 512)

        # Project CLIP → bottleneck
        self.clip_projection = nn.Sequential(
            nn.Linear(512, base_filters * 8),
            nn.ReLU(inplace=True),
            nn.Linear(base_filters * 8, base_filters * 8)
        )

        # -----------------------------
        # U-NET ENCODER
        # -----------------------------
        self.enc1 = self.conv_block(1, base_filters)
        self.enc2 = self.conv_block(base_filters, base_filters * 2)
        self.enc3 = self.conv_block(base_filters * 2, base_filters * 4)
        self.enc4 = self.conv_block(base_filters * 4, base_filters * 8)

        self.pool = nn.MaxPool2d(2)

        # -----------------------------
        # BOTTLENECK + FUSION
        # -----------------------------
        self.bottleneck = self.conv_block(base_filters * 8, base_filters * 16)

        self.fusion = nn.Sequential(
            nn.Conv2d(base_filters * 16 + base_filters * 8, base_filters * 16, 1),
            nn.BatchNorm2d(base_filters * 16),
            nn.ReLU(inplace=True),
            nn.Conv2d(base_filters * 16, base_filters * 16, 3, padding=1),
            nn.BatchNorm2d(base_filters * 16),
            nn.ReLU(inplace=True)
        )

        # -----------------------------
        # U-NET DECODER
        # -----------------------------
        self.up4 = nn.ConvTranspose2d(base_filters * 16, base_filters * 8, 2, 2)
        self.dec4 = self.conv_block(base_filters * 16, base_filters * 8)

        self.up3 = nn.ConvTranspose2d(base_filters * 8, base_filters * 4, 2, 2)
        self.dec3 = self.conv_block(base_filters * 8, base_filters * 4)

        self.up2 = nn.ConvTranspose2d(base_filters * 4, base_filters * 2, 2, 2)
        self.dec2 = self.conv_block(base_filters * 4, base_filters * 2)

        self.up1 = nn.ConvTranspose2d(base_filters * 2, base_filters, 2, 2)
        self.dec1 = self.conv_block(base_filters * 2, base_filters)

        # -----------------------------
        # CLIP-STYLE PIXEL CLASSIFIER
        # -----------------------------
        self.visual_proj = nn.Conv2d(base_filters, 512, kernel_size=1)

    def conv_block(self, in_c, out_c):
        return nn.Sequential(
            nn.Conv2d(in_c, out_c, 3, padding=1),
            nn.BatchNorm2d(out_c),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_c, out_c, 3, padding=1),
            nn.BatchNorm2d(out_c),
            nn.ReLU(inplace=True)
        )

    def forward(self, x_unet, x_clip=None):
        """
        x_unet: (B, 1, H, W)
        x_clip: (B, 3, 224, 224) or None
        """

        B, _, H, W = x_unet.shape

        # -----------------------------
        # CLIP FEATURES
        # -----------------------------
        if x_clip is not None:
            with torch.no_grad():
                clip_feat = self.clip_visual(x_clip)  # (B, 512)
            clip_feat = self.clip_projection(clip_feat)
            clip_feat = clip_feat.view(B, -1, 1, 1)
        else:
            clip_feat = torch.zeros(
                B, self.enc4[0].out_channels, 1, 1,
                device=x_unet.device
            )

        # -----------------------------
        # ENCODER
        # -----------------------------
        e1 = self.enc1(x_unet)
        e2 = self.enc2(self.pool(e1))
        e3 = self.enc3(self.pool(e2))
        e4 = self.enc4(self.pool(e3))

        # -----------------------------
        # BOTTLENECK + FUSION
        # -----------------------------
        b = self.bottleneck(self.pool(e4))

        clip_feat = clip_feat.expand(-1, -1, b.size(2), b.size(3))
        b = self.fusion(torch.cat([b, clip_feat], dim=1))

        # -----------------------------
        # DECODER
        # -----------------------------
        d4 = self.dec4(torch.cat([self.up4(b), e4], dim=1))
        d3 = self.dec3(torch.cat([self.up3(d4), e3], dim=1))
        d2 = self.dec2(torch.cat([self.up2(d3), e2], dim=1))
        d1 = self.dec1(torch.cat([self.up1(d2), e1], dim=1))

        # -----------------------------
        # CLIP-STYLE SEGMENTATION
        # -----------------------------
        visual = self.visual_proj(d1)           # (B, 512, H, W)
        visual = F.normalize(visual, dim=1)

        text = F.normalize(self.text_embeddings, dim=1)  # (C, 512)

        logits = torch.einsum("bchw,nc->bnhw", visual, text)

        return logits


In [13]:
# TEST
model_vlm = CLIPGuidedUNet(
    clip_model=clip_model,
    text_embeddings=text_embeddings,
    num_classes=3
).to(device)

x_unet = torch.randn(2, 1, 64, 64).to(device)
x_clip = torch.randn(2, 3, 224, 224).to(device)

y = model_vlm(x_unet, x_clip)

print("Output shape:", y.shape)


Output shape: torch.Size([2, 3, 64, 64])


Loss Functions et métriques


In [15]:
class DiceLoss(nn.Module):
    def __init__(self, smooth=1.0):
        super(DiceLoss, self).__init__()
        self.smooth = smooth
    
    def forward(self, pred, target):
        pred = torch.softmax(pred, dim=1)
        target_one_hot = torch.nn.functional.one_hot(target, num_classes=pred.shape[1])
        target_one_hot = target_one_hot.permute(0, 3, 1, 2).float()
        
        intersection = (pred * target_one_hot).sum(dim=(2, 3))
        union = pred.sum(dim=(2, 3)) + target_one_hot.sum(dim=(2, 3))
        
        dice = (2.0 * intersection + self.smooth) / (union + self.smooth)
        return 1 - dice.mean()


class VLMGuidedLoss(nn.Module):
    """
    Loss combinée pour VLM:
    - Dice Loss pour segmentation
    - Consistency loss pour alignement vision-texte
    """
    def __init__(self, weight_dice=0.8, weight_consistency=0.2):
        super(VLMGuidedLoss, self).__init__()
        self.dice_loss = DiceLoss()
        self.ce_loss = nn.CrossEntropyLoss()
        self.weight_dice = weight_dice
        self.weight_consistency = weight_consistency
    
    def forward(self, pred, target):
        dice = self.dice_loss(pred, target)
        ce = self.ce_loss(pred, target)
        
        # Consistency: encourage semantic coherence
        total_loss = self.weight_dice * dice + self.weight_consistency * ce
        
        return total_loss


def dice_score(pred, target, num_classes=3):
    pred = torch.softmax(pred, dim=1)
    pred = torch.argmax(pred, dim=1)
    
    dice_scores = []
    for c in range(num_classes):
        pred_c = (pred == c).float()
        target_c = (target == c).float()
        
        intersection = (pred_c * target_c).sum()
        union = pred_c.sum() + target_c.sum()
        
        if union == 0:
            dice_scores.append(1.0)
        else:
            dice_scores.append((2.0 * intersection / union).item())
    
    return dice_scores


def train_epoch_vlm(model, loader, criterion, optimizer, device, use_clip=True):
    model.train()
    total_loss = 0
    
    for batch in tqdm(loader, desc='Training VLM', leave=False):
        images_unet = batch['image_unet'].to(device)
        images_clip = batch['image_clip'].to(device) if use_clip and batch['image_clip'] is not None else None
        labels = batch['label'].to(device)
        
        optimizer.zero_grad()
        outputs = model(images_unet, images_clip)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    return total_loss / len(loader)


def validate_vlm(model, loader, criterion, device, use_clip=True):
    model.eval()
    total_loss = 0
    dice_scores_all = defaultdict(list)
    
    with torch.no_grad():
        for batch in tqdm(loader, desc='Validation VLM', leave=False):
            images_unet = batch['image_unet'].to(device)
            images_clip = batch['image_clip'].to(device) if use_clip and batch['image_clip'] is not None else None
            labels = batch['label'].to(device)
            
            outputs = model(images_unet, images_clip)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            
            scores = dice_score(outputs, labels)
            for i, score in enumerate(scores):
                dice_scores_all[f'class_{i}'].append(score)
    
    avg_dice = {k: np.mean(v) for k, v in dice_scores_all.items()}
    
    return total_loss / len(loader), avg_dice


print("✅ Loss functions et métriques VLM prêtes")

✅ Loss functions et métriques VLM prêtes


EXPÉRIENCE VLM-1 - CLIP-Guided U-Net

In [17]:
if CLIP_AVAILABLE:
    print("\n" + "🚀" * 35)
    print("EXPÉRIENCE VLM-1: CLIP-GUIDED U-NET")
    print("🚀" * 35)

    # =============================
    # CONFIGURATION
    # =============================
    NUM_EPOCHS_VLM = 10
    LEARNING_RATE_VLM = 1e-4
    WEIGHT_DECAY_VLM = 1e-5

    exp_vlm1_config = {
        'name': 'VLM1_CLIPGuidedUNet',
        'architecture': 'CLIP-Guided U-Net',
        'clip_model': 'ViT-B/32 (frozen)',
        'augmentation': 'Flip + Rotation',
        'loss': 'VLMGuidedLoss',
        'base_filters': 32
    }

    # =============================
    # MODEL
    # =============================
    model_vlm1 = CLIPGuidedUNet(
        clip_model=clip_model,
        text_embeddings=text_embeddings,
        num_classes=3,
        base_filters=32
    ).to(device)

    # Loss
    criterion_vlm1 = VLMGuidedLoss(
        weight_dice=0.8,
        weight_consistency=0.2
    )

    # Optimizer (CLIP frozen)
    trainable_params = [p for p in model_vlm1.parameters() if p.requires_grad]
    optimizer_vlm1 = optim.Adam(
        trainable_params,
        lr=LEARNING_RATE_VLM,
        weight_decay=WEIGHT_DECAY_VLM
    )

    # =============================
    # HISTORY
    # =============================
    history_vlm1 = {
        'train_loss': [],
        'val_loss': [],
        'val_dice': []
    }

    print(f"\n📊 Training VLM-1 for {NUM_EPOCHS_VLM} epochs")
    print("🔒 CLIP encoder: FROZEN")
    print("🔓 U-Net + Fusion layers: TRAINABLE")

    # =============================
    # TRAINING LOOP
    # =============================
    for epoch in range(NUM_EPOCHS_VLM):
        print(f"\nEpoch {epoch + 1}/{NUM_EPOCHS_VLM}")

        # ---- TRAIN (WITH CLIP) ----
        train_loss = train_epoch_vlm(
            model_vlm1,
            train_loader_vlm,
            criterion_vlm1,
            optimizer_vlm1,
            device,
            use_clip=True
        )
        history_vlm1['train_loss'].append(train_loss)

        # ---- VALIDATION (WITH CLIP) ----
        val_loss, val_dice = validate_vlm(
            model_vlm1,
            val_loader_vlm,
            criterion_vlm1,
            device,
            use_clip=True
        )
        history_vlm1['val_loss'].append(val_loss)
        history_vlm1['val_dice'].append(val_dice)

        avg_dice = np.mean(list(val_dice.values()))
        print(
            f"  Train Loss: {train_loss:.4f} | "
            f"Val Loss: {val_loss:.4f} | "
            f"Avg Dice: {avg_dice:.4f}"
        )

    # =============================
    # SAVE MODEL
    # =============================
    torch.save(
        model_vlm1.state_dict(),
        os.path.join(VLM_PATH, "vlm1_clip_guided_unet.pth")
    )

    final_dice_vlm1 = avg_dice
    print(f"\n✅ VLM-1 terminé")
    print(f"🎯 Final Dice Score (CLIP-Guided): {final_dice_vlm1:.4f}")

else:
    print("⚠️ CLIP non disponible — expérience VLM-1 ignorée")
    final_dice_vlm1 = None



🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀
EXPÉRIENCE VLM-1: CLIP-GUIDED U-NET
🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀

📊 Training VLM-1 for 10 epochs
🔒 CLIP encoder: FROZEN
🔓 U-Net + Fusion layers: TRAINABLE

Epoch 1/10


  Train Loss: 0.8099 | Val Loss: 0.8001 | Avg Dice: 0.7763

Epoch 2/10


  Train Loss: 0.7996 | Val Loss: 0.7992 | Avg Dice: 0.7483

Epoch 3/10


  Train Loss: 0.7993 | Val Loss: 0.7991 | Avg Dice: 0.8144

Epoch 4/10


  Train Loss: 0.7990 | Val Loss: 0.7988 | Avg Dice: 0.9174

Epoch 5/10


...

  Train Loss: 0.7988 | Val Loss: 0.7987 | Avg Dice: 0.9155

Epoch 7/10


  Train Loss: 0.7988 | Val Loss: 0.7987 | Avg Dice: 0.8889

Epoch 8/10


  Train Loss: 0.7988 | Val Loss: 0.7987 | Avg Dice: 0.8964

Epoch 9/10


  Train Loss: 0.7988 | Val Loss: 0.7987 | Avg Dice: 0.9145

Epoch 10/10


  Train Loss: 0.7988 | Val Loss: 0.7987 | Avg Dice: 0.9084

✅ VLM-1 terminé
🎯 Final Dice Score (CLIP-Guided): 0.9084
